<a href="https://colab.research.google.com/github/rexian/ML/blob/main/langchain/langchain_groq_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain-core langgraph>0.2.27
!pip install -qU "langchain[groq]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 4.0 MB/s eta 0:00:00


In [2]:
import os
import getpass

os.environ["GROQ_API_KEY"] = getpass.getpass("Enter Your Groq API Key:")

Enter Your Groq API Key:··········


In [3]:

import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("llama3-8b-8192", model_provider="groq")

In [4]:
prompt = "How do I become an AI Engineer?"
groq_response = model.invoke(prompt)
print(groq_response.content)

Becoming an AI Engineer requires a combination of education, skills, and experience in the fields of artificial intelligence, machine learning, and software engineering. Here are some steps to help you get started:

1. **Education**:
	* Earn a Bachelor's or Master's degree in a relevant field such as Computer Science, Electrical Engineering, Mathematics, or Statistics.
	* Online courses and certifications can also be beneficial, especially for those who already have a degree in a related field.
2. **Programming skills**:
	* Proficiency in at least one programming language, such as Python, Java, C++, or R.
	* Familiarity with machine learning libraries and frameworks, such as TensorFlow, PyTorch, or scikit-learn.
3. **Mathematics and statistics**:
	* Strong understanding of linear algebra, calculus, probability, and statistics.
	* Familiarity with mathematical concepts, such as optimization, graph theory, and dynamical systems.
4. **Machine learning and AI**:
	* Study machine learning a

# *Prompt Fundamentals in Language Models*

   - **Role of Prompts**: Set the stage for AI responses, from sentence completions to Q&A.
   - **Impact**: Determines how the AI will reply.

# *Designing Effective Prompts*

   - **Key Elements**: Clarity, context, user query, and a signal for AI to respond.

   - **Goal**: Direct AI towards the intended response.

# *Using Prompt Templates*
   - **Function**: Acts as a blueprint for crafting consistent, effective prompts.
   - **Advantage**: Ensures AI receives appropriate input for the expected output.

# *Simplicity in Explanation*
   - Concise and straightforward, making the concept easy to understand without deep technical details.

In [5]:
from langchain import PromptTemplate

# Define a simple prompt template

prompt_template = PromptTemplate.from_template("""
Human: What is the capital of {place}?
AI: The capital of {place} is {capital}
""")

prompt = prompt_template.format(place="California", capital="Sacramento")

print(prompt)


Human: What is the capital of California?
AI: The capital of California is Sacramento



In [6]:
# No Input Variable
no_input_prompt = PromptTemplate(input_variables=[], template="Tell me a joke.")
print(no_input_prompt.format())
response = model.invoke(no_input_prompt.format())
print(response.content)

# One Input Variable
one_input_prompt = PromptTemplate(input_variables=["adjective"], template="Tell me a {adjective} joke.")
print(one_input_prompt.format(adjective="funny"))

response = model.invoke(one_input_prompt.format(adjective="funny"))
print(response.content)


# Multiple Input Variables
multiple_input_prompt = PromptTemplate(
 input_variables=["adjective", "content"],
 template="Tell me a {adjective} joke about {content}."
)

multiple_input_prompt = multiple_input_prompt.format(adjective="funny", content="chickens")
print(multiple_input_prompt)
response = model.invoke(multiple_input_prompt)
print(response.content)

Tell me a joke.
Why couldn't the bicycle stand up by itself?

Because it was two-tired!
Tell me a funny joke.
Here's one:

Why couldn't the bicycle stand up by itself?

(wait for it...)

Because it was two-tired!

Hope that made you laugh!
Tell me a funny joke about chickens.
Here's one:

Why did the chicken go to the doctor?

Because it had a fowl cough!


In [7]:
prompt_template = PromptTemplate.from_template(
    template="Write a {length} story about: {content}"
)

prompt = prompt_template.format(
    length="2-sentence",
    content="Sunnyvale, CA, the Silicon Valley of the USA"
)

response = model.invoke(input=prompt)

print(response.content)

As the sun rose over the rows of sleek, modern homes in Sunnyvale, CA, software engineer Rachel stepped out of her front door to grab a coffee and catch up on the latest tech news, feeling grateful to call this hub of innovation her home. Little did she know, her latest project would soon be revolutionizing the way people communicated, cementing Sunnyvale's reputation as the Silicon Valley of the USA.


# Output parsers

- Output parsers shape the AI model output to an organized format
- They turn a block of text into organized data.
- They can guide the AI on how to format its responses for consistency and ease of use.

In [8]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain.output_parsers.list import ListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

prompt = PromptTemplate(
    template="List 5 {things}.\n{format_instructions}",
    input_variables=["things"],
    partial_variables={"format_instructions": format_instructions})

output = model.invoke(input=prompt.format(things="deep sea fishes"))

print(output.content)

output_parser.parse(str(output.content))

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`
Anglerfish, Viperfish, Fangtooth, Gulper Eel, Blobfish


['Anglerfish', 'Viperfish', 'Fangtooth', 'Gulper Eel', 'Blobfish']

# **LangChain Expression Language (LCEL) Overview**:
   - **Purpose**: Simplify building complex chains from basic components.
   - **Features**: Supports streaming, parallelism, and logging.

### Basic Use Case: Prompt + Model + Output Parser
   - **Common Approach**: Link a prompt template with a model.
   - **Chain Mechanism**: Using the `|` symbol, like a Unix pipe, to connect components.
   - **Process Flow**: User input → Prompt Template → Model → Output Parser.

### Understanding the Components
   - **Step-by-Step**:
     - User input is processed by the prompt template.
     - Prompt template's output goes to the model.
     - Model's output is refined by the output parser.
   - **Example Code**: `chain = prompt | model | output_parser` shows how to combine components into a single LCEL chain.

In [13]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("Write a bengali poem about {topic}")
model = init_chat_model("llama3-8b-8192", model_provider="groq")

output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "sunny days in Kolkata"})

"সূর্যাস্তেকালে কলকাতা\n\nসূর্য উদিত হয়েছে শহরের আকাশে\nকলকাতা সূর্যাস্তেকালে যথেষ্ট পরিবর্তন আনে\nশহরের রাস্তা সবুজ হয়ে উঠে\nপাখিরা সুর করে গায়\n\nসূর্যাস্তেকালে কলকাতা\nআকাশে মহাসমুদ্র প্রস্থিত আকাশ\nসূর্যাস্তেকালে কলকাতা\nসবুজ পার্কে সামাজিক সমাজ\n\nসূর্যাস্তেকালে কলকাতা\nশহরের রাস্তা সবুজ হয়ে উঠে\nকলকাতা সূর্যাস্তেকালে\nআকাশে মহাসমুদ্র প্রস্থিত আকাশ\n\nTranslation:\n\nSunny hours in Kolkata\n\nThe sun rises in the city's sky\nKolkata in the sunny hours undergoes a significant change\nThe city's roads turn green\nThe birds sing sweet melodies\n\nIn the sunny hours of Kolkata\nThe sky stretches out like the vast ocean\nIn the sunny hours of Kolkata\nThe social gatherings take place in the green parks\n\nIn the sunny hours of Kolkata\nThe city's roads turn green\nKolkata in the sunny hours\nThe sky stretches out like the vast ocean\n\nNote: Bengali is a highly inflected language with a complex grammar system, and direct translation may not always convey the same nuance and beauty 